**Урок 5. Spark on scala**

Условие: создайте csv файл с таким содержимым:

title,author,genre,sales,year<br/>
"1984", "George Orwell", "Science Fiction", 5000, 1949<br/>
"The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954<br/>
"To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960<br/>
"The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951<br/>
"The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925<br/>

Задание:

— Используя Spark прочитайте данные из файла csv.<br/>
— Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.<br/>
— Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.<br/>
— Отсортируйте данные по общему объему продаж в порядке убывания.<br/>
— Выведите результаты на экран.

In [ ]:
# Устанавливаем Scala
!apt-get install -y scala

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libhawtjni-runtime-java libjansi-java libjansi-native-java libjline2-java scala-library
  scala-parser-combinators scala-xml
Suggested packages:
  scala-doc
The following NEW packages will be installed:
  libhawtjni-runtime-java libjansi-java libjansi-native-java libjline2-java scala scala-library
  scala-parser-combinators scala-xml
0 upgraded, 8 newly installed, 0 to remove and 49 not upgraded.
Need to get 25.1 MB of archives.
After this operation, 28.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhawtjni-runtime-java all 1.17-1 [28.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjansi-native-java all 1.8-1 [23.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjansi-java all 1.18-1 [56.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/

In [ ]:
# Устанавливаем ядро Scala
!pip install spylon-kernel

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 18.5 MB/s eta 0:00:00
  Created wheel for spylon-kernel: filename=spylon_kernel-0.4.1-py2.py3-none-any.whl size=18339 sha256=13bf0af98b49d4f8f181cc59eb4d8e2ad40b13c51a50769a30a9f765050aeb60
  Stored in directory: /root/.cache/pip/wheels/f3/76/78/23b9e69e930f65ffd57dc9276c7ce33f9d130557ea8410a0ff
  Created wheel for spylon: filename=spylon-0.3.0-py3-none-any.whl size=136153 sha256=e03a2d65e4ae138d5740173eebb26010583a30463e01019def2c027648c4ad11
  Stored in directory: /root/.cache/pip/wheels/fe/63/90/775bb4496e281806c467ad6a52de4701cd042ddf1a249c2121
Successfully built spylon-kernel spylon


In [ ]:
# Добавляем Scala в список ядер
!python -m spylon_kernel install

[InstallKernelSpec] Installed kernelspec spylon-kernel in /usr/local/share/jupyter/kernels/spylon-kernel


In [ ]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}

val spark = SparkSession.builder.appName("Books").getOrCreate()

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@408223d6


In [ ]:
// Загрузка данных из файла

// Описание схемы данных
val schema = StructType(Seq(
  StructField("title", StringType, nullable = true),
  StructField("author", StringType, nullable = true),
  StructField("genre", StringType, nullable = true),
  StructField("sales", IntegerType, nullable = true),
  StructField("year", IntegerType, nullable = true)
))

val df = spark.read.schema(schema).option("header", "true").csv("/content/books.csv")
df.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|The Lord of the R...|     J.R.R. Tolkien|        Fantasy| 3000|1954|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|The Catcher in th...|      J.D. Salinger|          Novel| 2000|1951|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
+--------------------+-------------------+---------------+-----+----+



schema: org.apache.spark.sql.types.StructType = StructType(StructField(title,StringType,true),StructField(author,StringType,true),StructField(genre,StringType,true),StructField(sales,IntegerType,true),StructField(year,IntegerType,true))
df: org.apache.spark.sql.DataFrame = [title: string, author: string ... 3 more fields]


In [ ]:
// Фильтрация по объему продаж (оставить только книги, продажи которых превышают 3000 экземпляров)
val filteredDF = df.filter($"sales" > 3000)
filteredDF.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
+--------------------+-------------------+---------------+-----+----+



filteredDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [title: string, author: string ... 3 more fields]


In [ ]:
// Группировка по жанру и подсчет объема продаж
val aggregatedDF = df.groupBy("genre").agg(sum("sales").alias("total_sales")).orderBy(desc("total_sales"))
aggregatedDF.show()

+---------------+-----------+
|          genre|total_sales|
+---------------+-----------+
|          Novel|       6500|
|Science Fiction|       5000|
|Southern Gothic|       4000|
|        Fantasy|       3000|
+---------------+-----------+



aggregatedDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [genre: string, total_sales: bigint]


In [ ]:
// Сортировка данных по общему объему продаж в порядке убывания
val sortedDF = df.orderBy(desc("sales"))
sortedDF.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|The Lord of the R...|     J.R.R. Tolkien|        Fantasy| 3000|1954|
|The Catcher in th...|      J.D. Salinger|          Novel| 2000|1951|
+--------------------+-------------------+---------------+-----+----+



sortedDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [title: string, author: string ... 3 more fields]


In [ ]:
spark.stop()